In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
from qgsw.utils.sorting import sort_files
from qgsw.run_summary import RunSummary
from qgsw.models.qg.alpha import ChangingCoefficient
import torch
from matplotlib.axes import Axes
from qgsw.comparison.comparators import RMSE, absolute_difference
from qgsw.specs import DEVICE

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
%matplotlib widget

folder = ROOT.joinpath("output/g5k/pv_alpha_0_2")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
steps, files = sort_files(list(folder.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")
folder_1l = ROOT.joinpath("output/g5k/one_layer_baroclinic_100km")
run_1l = RunSummary.from_file(folder_1l.joinpath("_summary.toml"))
steps_1l, files_1l = sort_files(list(folder_1l.glob(f"{run_1l.configuration.model.prefix}*.npz")),run_1l.configuration.model.prefix,".npz")
folder_2l = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run_2l = RunSummary.from_file(folder_2l.joinpath("_summary.toml"))
steps_2l, files_2l = sort_files(list(folder_2l.glob(f"{run_2l.configuration.model.prefix}*.npz")),run_2l.configuration.model.prefix,".npz")

if not (run_1l.dt == run.dt == run_2l.dt):
    print("Timesteps are not matching.")
if not (run_1l.duration == run.duration == run_2l.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,18))

layer_1 = fig.add_subplot(3,3,1)
layer_2 = fig.add_subplot(3,3,2)
layer_3 = fig.add_subplot(3,3,3)
comparison_1 = fig.add_subplot(3,3,4)
comparison_2 = fig.add_subplot(3,3,5)
comparison_3 = fig.add_subplot(3,3,6)

rmses = fig.add_subplot(3,1,3)

rmse_1l_2l_30 = []
rmse_1l_30 = []
rmse_2l_30 = []

freq_save = len(files) // 10 + 1

for i in range(len(files)):
    if not (steps[i] == steps_1l[i] == steps_2l[i]):
        msg = f"Impossible to match steps {steps[i]},{steps_1l[i]} and {steps_2l[i]}."
        raise ValueError(msg)
    
    layer = torch.tensor(np.load(files[i])["omega"], device=DEVICE, dtype=torch.float64)[0,0]
    layer_1l = torch.tensor(np.load(files_1l[i])["omega"], device=DEVICE, dtype=torch.float64)[0,0]
    layer_2l = torch.tensor(np.load(files_2l[i])["omega"], device=DEVICE, dtype=torch.float64)[0,0]

    diff_1l_2l = np.power(layer_1l - layer_2l,2)
    diff_1l = np.power(layer_1l - layer,2)
    diff_2l = np.power(layer_2l - layer,2)

    rmse_1l_2l_30.append(RMSE(layer_1l, layer_2l))
    rmse_1l_30.append(RMSE(layer_1l, layer))
    rmse_2l_30.append(RMSE(layer, layer_2l))

    
    fig.suptitle(f"t: {steps[i]*run.dt}s/{steps[-1]*run.dt}s")

    layer_1.cla()
    layer_2.cla()
    layer_3.cla()
    comparison_1.cla()
    comparison_2.cla()
    comparison_3.cla()

    rmses.cla()

    layer_1.set_title("One Layer")
    layer_1.imshow(layer_1l)
    layer_2.set_title("Modified Layer")
    layer_2.imshow(layer)
    layer_3.set_title("Two Layers")
    layer_3.imshow(layer_2l)
    comparison_1.set_title(fr"1l - 2l: {rmse_1l_2l_30[-1]:.2f}")
    comparison_1.imshow(np.sqrt(diff_1l_2l))
    comparison_2.set_title(rf"1l - $\alpha$: {rmse_1l_30[-1]:.2f}")
    comparison_2.imshow(np.sqrt(diff_1l))
    comparison_3.set_title(fr"2l - $\alpha$: {rmse_2l_30[-1]:.2f}")
    comparison_3.imshow(np.sqrt(diff_2l))

    rmses.set_title("RMSE Evolution")
    rmses.set_xlim(steps[0], steps[-1])
    rmses.set_ylim(bottom=0, top=1.1*max(max(rmse_1l_2l_30), max(rmse_1l_30), max(rmse_2l_30)))
    rmses.plot(steps[:i+1], rmse_1l_2l_30, label = r"1l - 2l")
    rmses.plot(steps[:i+1], rmse_1l_30, label = r"1l - $\alpha$")
    rmses.plot(steps[:i+1], rmse_2l_30, label = r"2l - $\alpha$")
    rmses.legend()

    plt.pause(0.01)

plt.ioff()

In [ ]:
%matplotlib tk

folder_1 = ROOT.joinpath("output/g5k/pv_alpha_0")
run_1 = RunSummary.from_file(folder_1.joinpath("_summary.toml"))
steps_1, files_1 = sort_files(list(folder_1.glob(f"{run_1.configuration.model.prefix}*.npz")),run_1.configuration.model.prefix,".npz")
folder_2 = ROOT.joinpath("output/g5k/pvmixture_alpha_0")
run_2 = RunSummary.from_file(folder_2.joinpath("_summary.toml"))
steps_2, files_2 = sort_files(list(folder_2.glob(f"{run_2.configuration.model.prefix}*.npz")),run_2.configuration.model.prefix,".npz")

if not (run_2.dt == run_1.dt):
    print("Timesteps are not matching.")
if not (run_2.duration == run_1.duration):
    print("Duration are not matching.")

plt.ion()
fig = plt.figure(figsize = (18,12))

layer_1_axes:Axes = plt.subplot2grid((2,3),(0,0), fig=fig)
diff_axes:Axes = plt.subplot2grid((2,3),(0,1), fig=fig)
layer_2_axes:Axes = plt.subplot2grid((2,3),(0,2), fig=fig)

rmse_axes:Axes = plt.subplot2grid((2,3),(1,0), fig=fig, colspan=3)

rmses = []

freq_save = len(files_1) // 10 + 1

for i in range(len(files_1)):
    if not (steps_2[i] == steps_1[i]):
        msg = f"Impossible to match steps {steps_2[i]} and {steps_1[i]}."
        raise ValueError(msg)
    
    layer_1 = torch.tensor(np.load(files_1[i])["omega"], device=DEVICE, dtype=torch.float64)[0,0]
    layer_2 = torch.tensor(np.load(files_2[i])["omega"], device=DEVICE, dtype=torch.float64)[0,0]

    diff = absolute_difference(layer_1, layer_2)

    rmses.append(RMSE(layer_1, layer_2))

    
    fig.suptitle(f"t: {steps_1[i]*run_1.dt}s/{steps_1[-1]*run_1.dt}s")

    layer_1_axes.cla()
    layer_2_axes.cla()
    diff_axes.cla()

    rmse_axes.cla()

    layer_1_axes.set_title("1")
    layer_1_axes.imshow(layer_1)
    layer_2_axes.set_title("2")
    layer_2_axes.imshow(layer_2)
    diff_axes.set_title("|2-1|")
    diff_axes.imshow(diff)

    rmse_axes.set_title("RMSE Evolution")
    rmse_axes.set_xlim(steps_1[0], steps_1[-1])
    rmse_axes.set_ylim(bottom=0, top=1.1*max(rmses))
    rmse_axes.plot(steps_1[:i+1], rmses, label = r"RMSE 2-1", color='blue')
    rmse_axes.legend()

    plt.pause(0.01)

plt.ioff()